In [83]:
import requests
from bs4 import BeautifulSoup
import re
import json
import random
import os 

def get_news():
    url ='https://www.usatoday.com'
    r = requests.get(url)
    bs = BeautifulSoup(r.text, 'lxml')
    #lists = bs.select('ul.hfwmm-list.hfwmm-4uphp-list.hfwmm-light-list > li > a')
    lists = bs.select('div.gnt_m_th > a.gnt_m_th_a')

    for li in lists:
        href = url + li['href']
        r = requests.get(href)
        bs = BeautifulSoup(r.text, 'lxml')
        # texts = bs.select('div.asset-double-wide.double-wide.p402_premium > p.p-text')
        texts = bs.select('div.gnt_ar_b > p.gnt_ar_b_p')
        contents = [p.text for p in texts]
        contents = ' '.join(contents)
        return contents.lower()
    return None

# 영문 단어를 네이버 영어사전의 script api를 사용해 번역한다
def naver_translate(word):
    try:
        #url = f'https://ac.dict.naver.com/enendict/ac?q={word}&q_enc=utf-8&st=11001'
        url = f'https://ac-dict.naver.com/enko/ac?st=11&r_lt=11&q={word}'
        r = requests.get(url)
        j = json.loads(r.text)
        return (j['items'][0][0][2][0])
    except:
        # 뜻이 없는 경우
        return None

# 크롤링한 뉴스 기사에서 가장 많은 빈도수를 가진 단어를 추출하기 위해 정규식 생성
# 단어의 시작, 끝 글자가 a~z인 것중 4자리이상 15자리 이하의 단어를 추출
# a, the 등등의 의미없는 단어를 필터링
#          r은 역슬레쉬를 역슬레쉬로 인식하도록
def make_quiz(news):
    match_pattern = re.findall(r'\b[a-z]{4,15}\b', news)
    frequency = {}
    quiz_list = []

    for word in match_pattern:
        # get으로 키를 뽑았을 때 없다면 0으로 표시
        # word의 갯수 count를 가져온다
        count = frequency.get(word, 0)
        # {word: count}
        frequency[word] = count + 1

    for word, count in frequency.items():
        if count > 1:
            kor = naver_translate(word)
            
            if kor is not None:
                quiz_list.append({kor: word})

    #print(quiz_list)

#make_quiz(get_news())
def quiz():
    quiz_list = make_quiz(get_news())
   # quez_list2 = {k: v for k, v in quiz_list if k != ''}
    #print(quiz_list)
    try:
        random.shuffle(quiz_list)
    except TypeError:
        pass
    
    chance = 5
    count = 0
    
    for q in quiz_list:
        os.system('cls')
        kor = list(q.keys())[0]
        english = q.get(kor)
        
        print('*' * 90 )
        print(f'문제: {kor}')
        print('*' * 90 )
        
        for j in range(chance):
            user_input = str(input('위의 뜻이 의미하는 영문 단어를 입력하세요')).strip().lower()
            
            if user_input == english:
                print(f'정답입니다!! {len(count) - count}문제 남음')
                os.system('pause')
                break
            else:
                n = chance - (j + 1)
                if j == 0:
                    print(f'{user_input} 이/가 아닙니다. {n}번 기회가 남았습니다.')
                elif j == 1:
                    print(f'{user_input} 이/가 아닙니다. {n}번 기회가 남았습니다. 힌트: {english[0]}로 시작')
                elif j == 2:
                    hint = ' _ ' * int(len(english) - 2)
                    # Super --> su _ _ _
                    print(f'{user_input} 이/가 아닙니다. {n}번 기회가 남았습니다. 힌트: {english[0]} {english[1]} {hint}로 시작')
                elif j == 3:
                    hint = ' _ ' * int(len(english) - 3)
                    print(f'{user_input} 이/가 아닙니다. {n}번 기회가 남았습니다. 힌트: {english[0]} {english[1]} {english[2]} {hint}로 시작')
                else:
                    print(f'틀렸습니다! 정답은 {english}입니다.')
                    os.system('pause')
                    
    print('더 이상 문제가 없습니다.')
        
quiz()


TypeError: 'NoneType' object is not iterable